In [1]:
# LB Score = 0.83793

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV, StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler

In [3]:
df_train_features = pd.read_csv('data/train_features.csv', index_col='match_id_hash')
df_test_features = pd.read_csv('data/test_features.csv', index_col='match_id_hash')

In [4]:
df_targets = pd.read_csv('data/train_targets.csv')

In [5]:
split_index = df_train_features.shape[0]

In [6]:
df = pd.concat([df_train_features, df_test_features])

In [7]:
df_targets.head()

,match_id_hash,game_time,radiant_win,duration,time_remaining,next_roshan_team
0,a400b8f29dece5f4d266f49f1ae2e98a,155,False,992,837,NaN
1,b9c57c450ce74a2af79c9ce96fac144d,658,True,1154,496,NaN
2,6db558535151ea18ca70a6892197db41,21,True,1503,1482,Radiant
3,46a0ddce8f7ed2a8d9bd5edcbb925682,576,True,1952,1376,NaN
4,b1b35ff97723d9b7ade1c9c3cf48f770,453,False,2001,1548,NaN


In [8]:
y_train = df_targets['radiant_win']

## 2. Feature engineering

In [9]:
new_features = pd.DataFrame()

### 2.1 Kills features

In [10]:
r_kills_columns = [col for col in df if col.endswith('kills')][:5]
d_kills_columns = [col for col in df if col.endswith('kills')][5:]
new_features['r_kills_mean'] = df[r_kills_columns].mean(axis = 1)
new_features['r_kills_std'] = df[r_kills_columns].std(axis = 1)
new_features['r_kills_total'] = df[r_kills_columns].sum(axis = 1)
new_features['d_kills_mean'] = df[d_kills_columns].mean(axis = 1)
new_features['d_kills_std'] = df[d_kills_columns].std(axis = 1)
new_features['d_kills_total'] = df[d_kills_columns].sum(axis = 1)
new_features['kills_diff'] = new_features['d_kills_total'] - new_features['r_kills_total']

### 2.2 Gold features

In [11]:
r_gold_columns = [col for col in df if col.endswith('gold')][:5]
d_gold_columns = [col for col in df if col.endswith('gold')][5:]
new_features['r_gold_mean'] = df[r_gold_columns].mean(axis = 1)
new_features['r_gold_std'] = df[r_gold_columns].std(axis = 1)
new_features['r_gold_total'] = df[r_gold_columns].sum(axis = 1)
new_features['d_gold_mean'] = df[d_gold_columns].mean(axis = 1)
new_features['d_gold_std'] = df[d_gold_columns].std(axis = 1)
new_features['d_gold_total'] = df[d_gold_columns].sum(axis = 1)
new_features['gold_prop'] = new_features['d_gold_total'] / new_features['r_gold_total']
new_features['gold_prop1'] = new_features['r_gold_total'] / new_features['d_gold_total']

### 2.3 Stuns features

In [12]:
r_stuns_columns = [col for col in df if col.endswith('stuns')][:5]
d_stuns_columns = [col for col in df if col.endswith('stuns')][5:]

In [13]:
new_features['r_stuns_total'] = df[r_stuns_columns].sum(axis = 1)
new_features['d_stuns_total'] = df[d_stuns_columns].sum(axis = 1)

### 2.4 Max health features

In [14]:
r_max_health_columns = [col for col in df if col.endswith('max_health')][:5]
d_max_health_columns = [col for col in df if col.endswith('max_health')][5:]
new_features['d_max_health_total'] = df[r_max_health_columns].sum(axis = 1)
new_features['r_max_health_total'] = df[d_max_health_columns].sum(axis = 1)
new_features['max_health_prop'] = new_features['d_max_health_total'] / new_features['r_max_health_total']

### 2.5 Max mana features

In [15]:
r_max_mana_columns = [col for col in df if col.endswith('max_mana')][:5]
d_max_mana_columns = [col for col in df if col.endswith('max_mana')][5:]
new_features['d_max_mana_total'] = df[r_max_mana_columns].sum(axis = 1)
new_features['r_max_mana_total'] = df[d_max_mana_columns].sum(axis = 1)
new_features['max_mana_prop'] = new_features['r_max_mana_total'] / new_features['d_max_mana_total']

### 2.5 XP features

In [16]:
new_features['d_xp_total'] = df[[col for col in df if col.endswith('xp')][:5]].sum(axis = 1) + 1 # lets avoid division by zero
new_features['r_xp_total'] = df[[col for col in df if col.endswith('xp')][5:]].sum(axis = 1) + 1
new_features['d_xp_std'] = df[[col for col in df if col.endswith('xp')][:5]].std(axis = 1)
new_features['r_xp_std'] = df[[col for col in df if col.endswith('xp')][5:]].std(axis = 1)

In [17]:
new_features['xp_prop'] = new_features['d_xp_total'] / new_features['r_xp_total']

### 2.6 Observers

In [18]:
new_features['d_obs_total'] = df[[col for col in df if col.endswith('obs_placed')][:5]].sum(axis = 1)
new_features['r_obs_total'] = df[[col for col in df if col.endswith('obs_placed')][5:]].sum(axis = 1)
#new_features['obs_prop'] = new_features['d_obs_total']/ new_features['r_obs_total']

### 2.7 Towers killed

In [19]:
r_towers_killed_columns = [col for col in df if col.endswith('towers_killed')][:5]
d_towers_killed_columns = [col for col in df if col.endswith('towers_killed')][5:]
new_features['d_towers'] = df[d_towers_killed_columns].sum(axis = 1) + 1
new_features['r_towers'] = df[r_towers_killed_columns].sum(axis = 1) + 1
new_features['towers_prop'] = new_features['d_towers'] / new_features['r_towers']

### 2.8 Player level features

In [20]:
r_level_columns = [col for col in df if col.endswith('level')][:5]
d_level_columns = [col for col in df if col.endswith('level')][5:]
new_features['r_max_level'] = df[r_level_columns].max(axis = 1)
new_features['d_max_level'] = df[d_level_columns].max(axis = 1)
new_features['max_level_prop'] = new_features['d_max_level'] / new_features['r_max_level']

In [21]:
new_features['r_mean_level'] = df[r_level_columns].mean(axis = 1)
new_features['d_mean_level'] = df[d_level_columns].mean(axis = 1)
#new_features['mean_level_prop'] = new_features['r_mean_level'] / new_features['d_mean_level']

In [22]:
new_features['r_min_level'] = df[r_level_columns].min(axis = 1)
new_features['d_min_level'] = df[d_level_columns].min(axis = 1)
#new_features['min_level_prop'] = new_features['r_min_level'] / new_features['d_min_level']

### 2.9 Roshans

In [23]:
r_roshans_killes_columns = [col for col in df if col.endswith('roshans_killed')][:5]
d_roshans_killes_columns = [col for col in df if col.endswith('roshans_killed')][5:]
new_features['r_roshans'] = df[r_roshans_killes_columns].sum(axis = 1) + 1
new_features['d_roshans'] = df[d_roshans_killes_columns].sum(axis = 1) + 1
#new_features['roshans_prop'] = new_features['r_roshans'] / new_features['d_roshans']  <-- Seems to be a bad feature, dont know why

### 2.10 Coordinates

In [24]:
r_x_columns = [col for col in df if col.endswith('x')][:5]
d_x_columns = [col for col in df if col.endswith('x')][5:]

In [25]:
new_features['r_x_mean'] = df[r_x_columns].mean(axis = 1)
new_features['d_x_mean'] = df[d_x_columns].mean(axis = 1)
new_features['r_x_max'] = df[r_x_columns].max(axis = 1)
new_features['d_x_max'] = df[d_x_columns].max(axis = 1)
new_features['r_x_min'] = df[r_x_columns].min(axis = 1)
new_features['d_x_min'] = df[d_x_columns].min(axis = 1)

In [26]:
r_y_columns = [col for col in df if col.endswith('y')][:5]
d_y_columns = [col for col in df if col.endswith('y')][5:]

In [27]:
new_features['r_y_mean'] = df[r_y_columns].mean(axis = 1)
new_features['d_y_mean'] = df[d_y_columns].mean(axis = 1)

In [28]:
new_features['r_y_max'] = df[r_y_columns].max(axis = 1)
new_features['d_y_max'] = df[d_y_columns].max(axis = 1)
new_features['r_y_min'] = df[r_y_columns].min(axis = 1)
new_features['d_y_min'] = df[d_y_columns].min(axis = 1)

In [29]:
# coordinates of radiant start point
r_x_min = df[['r1_x', 'r2_x','r3_x','r4_x','r5_x']].min(axis = 1)
r_y_min = df[['r1_y', 'r2_y','r3_y','r4_y','r5_y']].min(axis = 1)

In [30]:
r_x_max = df[['r1_x', 'r2_x','r3_x','r4_x','r5_x']].max(axis = 1)
r_y_max = df[['r1_y', 'r2_y','r3_y','r4_y','r5_y']].max(axis = 1)

#### R distance from respown for each player

In [31]:
new_features['r1_dist'] = np.sqrt((df['r1_y'] - r_y_min)**2 + (df['r1_x'] - r_x_min)**2)
new_features['r2_dist'] = np.sqrt((df['r2_y'] - r_y_min)**2 + (df['r2_x'] - r_x_min)**2)
new_features['r3_dist'] = np.sqrt((df['r3_y'] - r_y_min)**2 + (df['r3_x'] - r_x_min)**2)
new_features['r4_dist'] = np.sqrt((df['r4_y'] - r_y_min)**2 + (df['r4_x'] - r_x_min)**2)
new_features['r5_dist'] = np.sqrt((df['r5_y'] - r_y_min)**2 + (df['r5_x'] - r_x_min)**2)

In [32]:
r_avg_dist = new_features[['r1_dist', 'r2_dist', 'r3_dist', 'r4_dist', 'r5_dist']].mean(axis = 1)

#### D distance from respown for each player

In [33]:
new_features['d1_dist'] = np.sqrt((df['d1_y'] - r_y_max)**2 + (df['d1_x'] - r_x_max)**2)
new_features['d2_dist'] = np.sqrt((df['d2_y'] - r_y_max)**2 + (df['d2_x'] - r_x_max)**2)
new_features['d3_dist'] = np.sqrt((df['d3_y'] - r_y_max)**2 + (df['d3_x'] - r_x_max)**2)
new_features['d4_dist'] = np.sqrt((df['d4_y'] - r_y_max)**2 + (df['d4_x'] - r_x_max)**2)
new_features['d5_dist'] = np.sqrt((df['d5_y'] - r_y_max)**2 + (df['d5_x'] - r_x_max)**2)

In [34]:
d_avg_dist = new_features[['d1_dist', 'd2_dist', 'd3_dist', 'd4_dist', 'd5_dist']].mean(axis = 1)

In [35]:
new_features['dist_prop'] =  d_avg_dist

In [36]:
#r_runes = df[[col for col in df if col.endswith('rune_pickups')][:5]].sum(axis = 1) + 1
#d_runes = df[[col for col in df if col.endswith('rune_pickups')][5:]].sum(axis = 1) + 1
#new_features['runes_prop'] = d_runes / r_runes

In [37]:
new_features.head()

,r_kills_mean,r_kills_std,r_kills_total,d_kills_mean,d_kills_std,d_kills_total,kills_diff,r_gold_mean,r_gold_std,r_gold_total,...,r2_dist,r3_dist,r4_dist,r5_dist,d1_dist,d2_dist,d3_dist,d4_dist,d5_dist,dist_prop
match_id_hash,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,0.0,0.000000,0,0.2,0.447214,1,1,407.4,85.902852,2037,...,67.201190,70.000000,100.000000,51.923020,89.106678,66.121101,72.691127,80.224684,50.477718,71.724262
b9c57c450ce74a2af79c9ce96fac144d,3.2,2.489980,16,0.6,0.547723,3,-13,3634.4,1153.576742,18172,...,88.022724,104.995238,104.804580,101.862653,68.117545,68.000000,38.832976,40.000000,26.000000,48.190104
6db558535151ea18ca70a6892197db41,0.0,0.000000,0,0.0,0.000000,0,0,176.0,0.000000,880,...,82.219219,66.000000,72.000000,64.000000,41.617304,63.529521,38.832976,39.446166,18.973666,40.479926
46a0ddce8f7ed2a8d9bd5edcbb925682,1.2,1.095445,6,0.2,0.447214,1,-5,2669.6,1077.473109,13348,...,60.000000,55.605755,58.137767,94.000000,82.097503,51.419841,98.183502,54.000000,94.191295,75.978428
b1b35ff97723d9b7ade1c9c3cf48f770,0.4,0.547723,2,0.8,0.836660,4,2,1615.8,300.503245,8079,...,72.000000,50.477718,60.000000,96.187317,90.088845,60.000000,60.959003,58.034473,70.114193,67.839303


In [38]:
#columns = df.columns[-53:]

In [39]:
# These features decrease the score for RF, but we will use them in the more sophisticated models
new_features = new_features.drop(['r1_dist', 'r2_dist', 'r3_dist','r4_dist','r5_dist',
                                  'd1_dist', 'd2_dist', 'd3_dist','d4_dist','d5_dist',
                                  'd_xp_total'], axis = 1)

In [40]:
X_train = new_features[:split_index]

In [41]:
X_test = new_features[split_index:]

In [42]:
y_train = df_targets['radiant_win'] * 1

In [43]:
X_train.shape, y_train.shape

((39675, 54), (39675,))

In [44]:
rf = RandomForestClassifier(n_estimators=100, max_depth=15, min_samples_leaf = 5, random_state=17)

In [45]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=15, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=17, verbose=0,
                       warm_start=False)

In [46]:
cross_val_score(rf, X_train, y_train, cv = 5, n_jobs = -1, scoring='roc_auc').mean()

0.8183922508859706

In [47]:
# 0.8189803655571639 - max level prop

In [48]:
# 0.8185534881414543 - max tower prop

In [49]:
# 0.8184250096324922 - max mana prop

In [50]:
# 0.8182711584824215

In [51]:
# 0.8184619965576416 - max health proportion

In [52]:
# 0.8181779266124078 - xp_gold removed

In [53]:
# 0.8180850738287315 - xp_proportion

In [54]:
#0.8138925440421548 - d_dist

In [55]:
# 0.8124434204241302

In [56]:
def prepare_submit(model, x_test, file_name = 'submission'):
    y_test_pred= model.predict_proba(X_test)[:, 1]
    df_submission = pd.DataFrame({'radiant_win_prob': y_test_pred}, 
                                 index=df_test_features.index)
    df_submission.to_csv('first_pred_all_data.csv')